

# Nettoyage de la base de données

### <u>1. Description de la base de données

**Les données** </u>

La base de données utilisée est celle de Data.gouv qui liste les festivals en France. Les critères de sélection des festivals sont les suivants :
- avoir eu lieu en 2019
- avoir connu au moins deux éditions en 2019
- se dérouler sur plus d’une journée
- compter au moins 5 spectacles, représentations, concerts ou projections
<u> 



**Description de la base de données** </u>

La base de donnée regroupe les éléments suivants :
- Identité du festival : nom, disciplines dominantes, sous-catégorie au sein des disciplines
- Données géographiques : région de déroulement, département, adresse postale, géocodage xy (coordonnées)
- Données administratives : identifiant, code insee...
- Données temporelles : période de déroulement, date de création...
- Trouver/communiquer avec le festival : site internet et adresse mail.

### <u>2. Suppression des données inutiles 
</u>

On a choisit de supprimer les colonnes suivantes : 
- Code postal (de la commune principale de déroulement)
- Code Insee commune
- Code Insee EPCI
- Libellé EPCI	
- Numéro de voie	 
- Type de voie (rue, Avenue, boulevard, etc.)	
- Nom de la voie	
- Adresse postale	
- Complément d'adresse (facultatif)
- Décennie de création du festival
- Année de création du festival
- Identifiant Agence A
- identifiant CNM

On a aussi fait le choix de ne conserver que les festivals se déroulant en France métropolitaine. 


In [29]:
import pandas as pd


# Charger le fichier CSV
df = pd.read_csv('festivals_en_France.csv', sep=';', encoding='utf-8-sig')

# Afficher les premières lignes pour vérifier le contenu
print(df.head())


# Supprimer les colonnes qui ne nous intéressent pas
df = df.drop(["Code postal (de la commune principale de déroulement)", "Code Insee commune", "Code Insee EPCI", "Libellé EPCI", "Numéro de voie", "Type de voie (rue, Avenue, boulevard, etc.)", "Nom de la voie", "Adresse postale", "Complément d'adresse (facultatif)", "Décennie de création du festival", "Année de création du festival", "Identifiant Agence A", "identifiant CNM"], axis=1)

# Afficher les colonnes restantes pour vérifier
print(df.columns.tolist())


# Supprimer les festivals hors France métropolitaine
regions_non_metropolitaines = ["Guadeloupe", "Martinique", "Guyane", "La Réunion", "Mayotte", "Polynésie française", "Saint-Pierre-et-Miquelon", "Saint-Barthélemy"]
df = df[~df["Région principale de déroulement"].isin(regions_non_metropolitaines)]

# Vérifier le contenu après suppression
print(df["Région principale de déroulement"].unique())


            Nom du festival Envergure territoriale  \
0          Festival andalou              Régionale   
1  Festival Rêves d'enfants                    NaN   
2      Printemps de Paroles                    NaN   
3                 Pharaonic                    NaN   
4           Sarcus Festival                    NaN   

  Région principale de déroulement Département principal de déroulement  \
0       Provence-Alpes-Côte d'Azur                             Vaucluse   
1       Provence-Alpes-Côte d'Azur                         Hautes-Alpes   
2                    Île-de-France                       Seine-et-Marne   
3             Auvergne-Rhône-Alpes                               Savoie   
4              Centre-Val de Loire                       Indre-et-Loire   

  Commune principale de déroulement  \
0                           Avignon   
1                          Briançon   
2                Bussy-Saint-Martin   
3                          Chambéry   
4              Chemillé-sur-I

### <u>3. Modification et mise en forme des données  

**Transformation des données** </u>

Transformation des données sous forme de listes de listes. La première clef utilisée est nom du festival_identifiant car il y a des festivals ayant le même nom mais se déroulant sur des communes différentes. Ajouter l'identifiant permet d'avoir une clef unique. 

On veut quelque chose de cette forme : {"nom-festival_id" : {"nom-festival:..., "date":..., }}, {"nom-festival_id2" : {...} }, etc. 





In [30]:
# Construire un dictionnaire où chaque clé principale est "Nom du festival_Identifiant"
dictionnaire = {}
for _, row in df.iterrows():
    # Utiliser la colonne "Identifiant" du DataFrame comme partie de la clé
    cle_principale = f"{row['Nom du festival']}_{row['Identifiant']}"
    
    # Ajouter un sous-dictionnaire contenant toutes les colonnes, sauf la colonne "Identifiant"
    dictionnaire[cle_principale] = row.drop("Identifiant").to_dict()

# Afficher un exemple
list(dictionnaire.items())[:5]  # Afficher les 5 premières entrées du dictionnaire pour vérifier


[('Festival andalou_FEST_84007_514',
  {'Nom du festival': 'Festival andalou',
   'Envergure territoriale': 'Régionale',
   'Région principale de déroulement': "Provence-Alpes-Côte d'Azur",
   'Département principal de déroulement': 'Vaucluse',
   'Commune principale de déroulement': 'Avignon',
   'Site internet du festival': 'https://lefestivalandalou.com/',
   'Adresse e-mail': 'contact@lefestivalandalou.com',
   'Discipline dominante': 'Musique',
   'Sous-catégorie spectacle vivant': nan,
   'Sous-catégorie musique': 'Musiques du monde',
   'Sous-catégorie Musique CNM': '09- Pluridisciplinaire',
   'Sous-catégorie cinéma et audiovisuel': nan,
   'Sous-catégorie arts visuels et arts numériques': nan,
   'Sous-catégorie livre et littérature': nan,
   'Période principale de déroulement du festival': 'Avant-saison (1er janvier - 20 juin)',
   'Géocodage xy': '43.9352448339, 4.84071572505'}),
 ("Festival Rêves d'enfants_FEST_05023_1069",
  {'Nom du festival': "Festival Rêves d'enfants",


<u>

**La catégorie "pluridisciplinaire"**  </u>

Il s'agit ici d'expliciter la catégorie "pluridisciplinaire". En effet, l'utilisateur va cocher le type de festival qu'il souhaite : il faut donc spécifier à quelles disciplines appartient le festival pour que celui-ci apparaisse lorsque l'utilisateur cochera. Ainsi, si la colonne "Sous-catégorie [...]" n'est pas vide, on renomme la colonne "Discipline dominante" avec le nom de la sous-catégorie.

**Par exemple** : le festival "Rêves d'enfants" est noté pluridisciplinaire et est un festival de littérature et de spectacle vivant. Il faut qu'il apparaisse lorsque l'utilisateur coche "Littérature" ou "Spectacle vivant". On renomme ainsi la colonne "Discipline dominante" "Livre, littérature et Spectacle vivant". 


Il reste néanmoins 290 festivals marqués comme "pluridisciplinaire" mais sans sous-catégorie renseignée. Nous avons donc choisi de supprimer ces festivals pour deux raisons : 
1. Dans le questionnaire, l'utilisateur clique sur une catégorie de festival qu'il souhaite, si le festival n'en a pas il ne peut cliquer dessus. 
2. Il y a trop de festivals dans ce cas (290) pour pouvoir ajouter manuellement les catégories en question. 

**Par exemple**, le festival Printemps de paroles est noté pluridisciplinaire, mais avec aucune sous-catégorie associée.

In [33]:
# Liste des colonnes sous-catégories que l'on analyse
colonnes_sous_categories = [
    "Sous-catégorie spectacle vivant",
    "Sous-catégorie musique",
    "Sous-catégorie cinéma et audiovisuel",
    "Sous-catégorie arts visuels et arts numériques",
    "Sous-catégorie livre et littérature"
]

# Dictionnaire pour associer les colonnes de sous-catégorie à leurs noms dans "Discipline dominante"
mapping_discipline = {
    "Sous-catégorie spectacle vivant": "Spectacle vivant",
    "Sous-catégorie musique": "Musique",
    "Sous-catégorie cinéma et audiovisuel": "Cinéma et audiovisuel",
    "Sous-catégorie arts visuels et arts numériques": "Arts visuels et numériques",
    "Sous-catégorie livre et littérature": "Livre et littérature"
}

# Fonction pour renommer la discipline dominante en respectant les correspondances spécifiques
def renommer_discipline(row):
    # Si "Discipline dominante" est pluridisciplinaire
    if row["Discipline dominante"] == "Pluridisciplinaire":
        # Liste des disciplines à partir des sous-catégories non vides
        sous_categories = [mapping_discipline[col] for col in colonnes_sous_categories if pd.notna(row[col])]
        # Si au moins une sous-catégorie est trouvée, les utiliser
        if sous_categories:
            return " et ".join(sous_categories)
        else:
            return "A supprimer"
    # Sinon, garder la valeur d'origine
    return row["Discipline dominante"]

# Appliquer la fonction sur le DataFrame
df["Discipline dominante"] = df.apply(renommer_discipline, axis=1)

# Supprimer les festivals marqués "A supprimer"
df = df[df["Discipline dominante"] != "A supprimer"]

# Vérifier les résultats
print(df[["Nom du festival", "Discipline dominante"]].head(10))

                     Nom du festival                      Discipline dominante
0                   Festival andalou                                   Musique
1           Festival Rêves d'enfants  Spectacle vivant et Livre et littérature
3                          Pharaonic                                   Musique
4                    Sarcus Festival                                   Musique
5                   Jazz in Fougères                                   Musique
6                           L'aparté                          Spectacle vivant
7   Art contemporain en milieu rural             Arts visuels, arts numériques
8       Les Musicales du Noirmoutier                                   Musique
10                        Ai que Bom                                   Musique
11                Printemps des Rues                          Spectacle vivant


<u>

**Modification des sous-catégories** </u>

Pour plus de clarté, nous avons choisis de modifier les sous-catégories présentes dans la base de données qui étaient beaucoup trop nombreuses et les avons réduits à 10 par catégorie. 

In [36]:
# Dictionnaire de regroupement des sous-catégories en 10 catégories principales
regroupements_spectacle_vivant = {
    "Théâtre": ["Théâtre", "Théâtre, Danse", "Théâtre, Marionnettes", "Théâtre, arts du conte", "Arts du conte"],
    "Danse": ["Danse contemporaine", "Danse traditionnelle", "Danse, théâtre", "Danse"],
    "Arts de la Rue": ["Arts de la Rue", "Spectacles de rue", "Arts de la rue et Cirque"],
    "Cirque": ["Arts du Cirque", "Nouveau Cirque", "Arts de la piste", "Cirque"],
    "Musique et Chant": ["Musique", "Chanson", "Concerts", "Opéra"],
    "Marionnettes et Théâtre d'objets": ["Marionnettes", "Théâtre d'objets", "Marionnettes et Théâtre visuel", "Arts de la marionnette", "Marionnettes et théâtre d'objet"],
    "Spectacles pour Jeune Public": ["Jeune Public", "Spectacle pour enfants", "Conte musical", "Théâtre jeune public"],
    "Performance et Arts Visuels": ["Performance", "Arts visuels", "Exposition", "Cinéma et audiovisuel", "Magie"],
    "Humour et Café-Théâtre": ["Humour", "Café-Théâtre", "Théâtre d'humour"],
    "Pluridisciplinaire": ["Pluridisciplinaire", "Spectacle vivant pluridisciplinaire"]
}

# Inverser le dictionnaire pour associer chaque sous-catégorie à une catégorie principale
inverse_regroupements = {}
for categorie, sous_cats in regroupements_spectacle_vivant.items():
    for sous_cat in sous_cats:
        inverse_regroupements[sous_cat.lower()] = categorie  # Utilisation de `.lower()` pour la cohérence

# Fonction pour attribuer les nouvelles sous-catégories avec gestion des NaN
def attribuer_sous_categories(row):
    # Vérifier si la colonne est vide
    if pd.isna(row["Sous-catégorie spectacle vivant"]):
        return None  # Pas de sous-catégorie si NaN
    
    # Diviser les sous-catégories multiples
    sous_categories = str(row["Sous-catégorie spectacle vivant"]).split(", ")
    nouvelles_categories = set()
    
    for sous_cat in sous_categories:
        sous_cat_normalise = sous_cat.lower().strip()  # Normaliser en minuscule et retirer les espaces inutiles
        if sous_cat_normalise in inverse_regroupements:
            nouvelles_categories.add(inverse_regroupements[sous_cat_normalise])
        else:
            # Débogage : Afficher les sous-catégories non reconnues
            print(f"Sous-catégorie non reconnue : '{sous_cat}' pour le festival '{row['Nom du festival']}'")
    
    return ", ".join(nouvelles_categories) if nouvelles_categories else None

# Appliquer la fonction corrigée sur le DataFrame
df["Nouvelles sous-catégories spectacle vivant"] = df.apply(attribuer_sous_categories, axis=1)

# Afficher les festivals ayant des valeurs manquantes pour vérifier
print("Festivals avec des sous-catégories manquantes :")
print(df[df["Sous-catégorie spectacle vivant"].isna()][["Nom du festival", "Sous-catégorie spectacle vivant"]])

# Vérification des résultats pour le festival de la Haute Romanche
resultat = df[df["Nom du festival"] == "Haute Romanche"]
print("Résultat pour le festival de la Haute Romanche :")
print(resultat[["Nom du festival", "Sous-catégorie spectacle vivant", "Nouvelles sous-catégories spectacle vivant"]])



Sous-catégorie non reconnue : 'Conte' pour le festival 'Thoiry 100 Histoires'
Sous-catégorie non reconnue : 'Conte' pour le festival 'Conte en campagne'
Sous-catégorie non reconnue : 'Arts de la piste ; arts de la rue ; théâtre' pour le festival 'Les rencontres de la haute romanche'
Sous-catégorie non reconnue : 'Cirque et Arts de la rue' pour le festival 'Renc'arts de Rue'
Sous-catégorie non reconnue : 'Pluridisciplinaire culture' pour le festival 'Marsan Sur Scènes'
Sous-catégorie non reconnue : 'Pluridisciplinaire culture' pour le festival 'Quartiers d'été'
Sous-catégorie non reconnue : 'Arts de la piste ; Arts de la rue ; Théâtre' pour le festival 'La Semaine du Cirque'
Sous-catégorie non reconnue : 'Spectacle vivant' pour le festival 'Les Hivernales'
Sous-catégorie non reconnue : 'Arts du théâtre ; danse ; musique' pour le festival 'Journées des arts vivants'
Sous-catégorie non reconnue : 'Théâtre d'humour/café-théâtre' pour le festival 'Les folklories internationales'
Sous-catégo

In [ ]:
import re  # Importer le module pour gérer les séparateurs multiples

# Étape 1 : Dictionnaire élargi
regroupements_spectacle_vivant = {
    "Théâtre": ["Théâtre", "Théâtre - humour", "Théâtre ; Lecture publique", "Arts du théâtre"],
    "Danse": ["Danse", "Danses traditionnelles"],
    "Arts de la Rue": ["Arts de la Rue", "Cirque et Arts de la rue", "Arts de la rue ; Théâtre"],
    "Cirque": ["Arts de la piste", "Nouveau Cirque", "Arts de la piste ; Arts de la rue ; Théâtre"],
    "Musique et Chant": ["Musique", "Musiques traditionnelles", "Chanson", "Concerts", "Opéra"],
    "Marionnettes et Théâtre d'objets": ["Théâtre d'objet", "Marionnettes"],
    "Spectacles pour Jeune Public": ["Conte", "Jeune Public", "Conte musical"],
    "Performance et Arts Visuels": ["Performance", "Arts visuels", "photo", "vidéo", "Exposition"],
    "Humour et Café-Théâtre": ["Humour", "Théâtre d'humour/café-théâtre"],
    "Pluridisciplinaire": ["Pluridisciplinaire", "Pluridisciplinaire culture", "Pluridisciplinaire à dominante spectacle vivant"],
    "Non spécifié":[None]
}

# Étape 2 : Inverser le dictionnaire
inverse_regroupements = {}
for categorie, sous_cats in regroupements_spectacle_vivant.items():
    for sous_cat in sous_cats:
        inverse_regroupements[sous_cat.lower()] = categorie

# Étape 3 : Fonction corrigée
def attribuer_sous_categories(row):
    # Vérifier si la colonne est vide
    if pd.isna(row["Sous-catégorie spectacle vivant"]):
        return None
    
    # Diviser les sous-catégories multiples avec plusieurs séparateurs
    sous_categories = re.split(r"[;/,]", str(row["Sous-catégorie spectacle vivant"]))
    nouvelles_categories = set()
    
    for sous_cat in sous_categories:
        sous_cat_normalise = sous_cat.lower().strip()  # Normaliser en minuscule et retirer les espaces
        if sous_cat_normalise in inverse_regroupements:
            nouvelles_categories.add(inverse_regroupements[sous_cat_normalise])
        else:
            # Débogage pour les sous-catégories non reconnues
            print(f"Sous-catégorie non reconnue : '{sous_cat}' pour le festival '{row['Nom du festival']}'")
    
    return ", ".join(nouvelles_categories) if nouvelles_categories else None

# Étape 4 : Appliquer la fonction
df["Nouvelles sous-catégories spectacle vivant"] = df.apply(attribuer_sous_categories, axis=1)

# Afficher un aperçu
print(df[["Nom du festival", "Sous-catégorie spectacle vivant", "Nouvelles sous-catégories spectacle vivant"]].head(20))


Sous-catégorie non reconnue : 'Arts du cirque' pour le festival 'Festival Cirq'ô champs'
Sous-catégorie non reconnue : 'Spectacle vivant pluridisciplinaire' pour le festival 'Festival Tête en L’R'
Sous-catégorie non reconnue : 'Spectacle vivant pluridisciplinaire' pour le festival 'Les Odyssées'
Sous-catégorie non reconnue : 'Arts de la marionnette' pour le festival 'Solstice de la marionnette'
Sous-catégorie non reconnue : 'Marionnettes et théâtre d'objet' pour le festival 'Géo Condé'
Sous-catégorie non reconnue : 'Arts du conte' pour le festival 'Amies Voix'
Sous-catégorie non reconnue : 'Arts du conte' pour le festival 'Petite marée - Contes pour les tout-petits'
Sous-catégorie non reconnue : 'Cirque' pour le festival 'Utopistes'
Sous-catégorie non reconnue : 'Arts du cirque' pour le festival 'Fivestival'
Sous-catégorie non reconnue : 'Spectacle vivant pluridisciplinaire' pour le festival 'La Cabanne des Arts'
Sous-catégorie non reconnue : 'Cirque' pour le festival 'Mourenx Fait Son

In [39]:
df.head()


,Nom du festival,Envergure territoriale,Région principale de déroulement,Département principal de déroulement,Commune principale de déroulement,Site internet du festival,Adresse e-mail,Discipline dominante,Sous-catégorie spectacle vivant,Sous-catégorie musique,Sous-catégorie Musique CNM,Sous-catégorie cinéma et audiovisuel,Sous-catégorie arts visuels et arts numériques,Sous-catégorie livre et littérature,Période principale de déroulement du festival,Identifiant,Géocodage xy,Nouvelles sous-catégories spectacle vivant
0,Festival andalou,Régionale,Provence-Alpes-Côte d'Azur,Vaucluse,Avignon,https://lefestivalandalou.com/,contact@lefestivalandalou.com,Musique,NaN,Musiques du monde,09- Pluridisciplinaire,NaN,NaN,NaN,Avant-saison (1er janvier - 20 juin),FEST_84007_514,"43.9352448339, 4.84071572505",None
1,Festival Rêves d'enfants,NaN,Provence-Alpes-Côte d'Azur,Hautes-Alpes,Briançon,www.serre-chevalier.com,info@ot-briancon.fr,Spectacle vivant et Livre et littérature,Théâtre,NaN,NaN,NaN,NaN,Fiction,Avant-saison (1er janvier - 20 juin),FEST_05023_1069,"44.8994986041, 6.64947524018",Théâtre
3,Pharaonic,NaN,Auvergne-Rhône-Alpes,Savoie,Chambéry,http://www.pharaonic.fr/,NaN,Musique,NaN,Musiques amplifiées ou électroniques,02- Musiques amplifiées ou électroniques,NaN,NaN,NaN,Avant-saison (1er janvier - 20 juin),FEST_73065_1434,"45.583182552, 5.90903392417",None
4,Sarcus Festival,NaN,Centre-Val de Loire,Indre-et-Loire,Chemillé-sur-Indrois,https://sarcus.fr,contact@bleuenuit.fr,Musique,NaN,"Musiques électroniques, techno",02- Musiques amplifiées ou électroniques,NaN,NaN,NaN,Après-saison (6 septembre - 31 décembre),FEST_37069_1605,"47.1535504221, 1.15871292042",None
5,Jazz in Fougères,NaN,Bretagne,Ille-et-Vilaine,Fougères,www.le-coquelicot.fr,jazzetjava@gmail.com,Musique,NaN,NaN,"03- Jazz, blues et musiques improvisées",NaN,NaN,NaN,Avant-saison (1er janvier - 20 juin),FEST_35115_2297,"48.3524697115, -1.19431177241",None
